# Create your custom pathlib class
> Utils

- toc: true 
- badges: true
- comments: true
- categories: [fastcore, fastai, python]

This is the fourth of a series of posts whose aim is to go through the complete development of [fastcore](http://fastcore.fast.ai/) and [fastai2](https://dev.fast.ai/).

In [1]:
#hide
from types import FunctionType
from copy import copy
import functools
from pathlib import Path
import tempfile
from fastcore.test import test_eq
import pickle
import io
from fastcore.utils import L
import mimetypes
import itertools

## Recap: patching an existing class (adding a function to it)

Here we do a recap of a functionality covered in this post

First, we need a way to properly copy a function. i.e. create an exact replica of a given function.

In [2]:
def copy_func(f):
    "Copy a non-builtin function (NB `copy.copy` does not work for this)"
    if not isinstance(f,FunctionType): return copy(f)
    fn = FunctionType(f.__code__, f.__globals__, f.__name__, f.__defaults__, f.__closure__)
    fn.__dict__.update(f.__dict__)
    return fn

Here we copy the function for each class in `cls`, and we also copy its attributes in `functools.WRAPPER_ASSIGNMENTS`, i.e., `('__module__', '__name__', '__qualname__', '__doc__', '__annotations__')`. `__qualname__` needs to be updated to match its new location as a method (or property depending on `as_prop`) in the class beiing considered.

In [3]:
def patch_to(cls, as_prop=False):
    "Decorator: add `f` to `cls`"
    if not isinstance(cls, (tuple,list)): cls=(cls,)
    def _inner(f):
        for c_ in cls:
            nf = copy_func(f)
            for o in functools.WRAPPER_ASSIGNMENTS: setattr(nf, o, getattr(f,o))
            nf.__qualname__ = f"{c_.__name__}.{f.__name__}"
            setattr(c_, f.__name__, property(nf) if as_prop else nf)
        return f
    return _inner

Same as above, but infers the class by the first type annotation of `f`.

In [4]:
def patch(f):
    "Decorator: add `f` to the first parameter's class (based on f's type annotations)"
    cls = next(iter(f.__annotations__.values()))
    return patch_to(cls)(f)

Now that we have a decorator to start patching classes, let's go ahead and add some methods to `Path`.

In [5]:
@patch
def readlines(self:Path, hint=-1, encoding='utf8'):
    "Read the content of `fname`"
    with self.open(encoding=encoding) as f: return f.readlines(hint)

In [6]:
@patch
def read(self:Path, size=-1, encoding='utf8'):
    "Read the content of `fname`"
    with self.open(encoding=encoding) as f: return f.read(size)

In [7]:
@patch
def write(self:Path, txt, encoding='utf8'):
    "Write `txt` to `self`, creating directories as needed"
    self.parent.mkdir(parents=True,exist_ok=True)
    with self.open('w', encoding=encoding) as f: f.write(txt)

In [8]:
with tempfile.NamedTemporaryFile() as f:
    fn = Path(f.name)
    fn.write('t')
    t = fn.read()
    test_eq(t,'t')
    t = fn.readlines()
    test_eq(t,['t'])

In [9]:
@patch
def save(fn:Path, o):
    "Save a pickle file, to a file name or opened file"
    if not isinstance(fn, io.IOBase): fn = open(fn,'wb')
    try: pickle.dump(o, fn)
    finally: fn.close()

In [10]:
@patch
def load(fn:Path):
    "Load a pickle file from a file name or opened file"
    if not isinstance(fn, io.IOBase): fn = open(fn,'rb')
    try: return pickle.load(fn)
    finally: fn.close()

In [11]:
with tempfile.NamedTemporaryFile() as f:
    fn = Path(f.name)
    fn.save('t')
    t = fn.load()
test_eq(t,'t')

`L` was covered in [this](https://juan-carlos-calvo.github.io/blog/fastcore/fastai/python/2020/04/25/create-your-custom-list-type.html) post and can be imported with `from fastcore.utils import L`

In [12]:
@patch
def ls(self:Path, n_max=None, file_type=None, file_exts=None):
    "Contents of path as a list"
    extns=L(file_exts)
    if file_type: extns += L(k for k,v in mimetypes.types_map.items() if v.startswith(file_type+'/'))
    has_extns = len(extns)==0
    res = (o for o in self.iterdir() if has_extns or o.suffix in extns)
    if n_max is not None: res = itertools.islice(res, n_max)
    return L(res)

We add an `ls()` method to `pathlib.Path` which is simply defined as `list(Path.iterdir())`, mainly for convenience in REPL environments such as notebooks.

In [13]:
path = Path()
t = path.ls()
assert len(t)>0
t1 = path.ls(10)
test_eq(len(t1), 10)
t2 = path.ls(file_exts='.ipynb')
assert len(t)>len(t2)
t[0]

Path('02_utils.ipynb')

You can also pass an optional `file_type` MIME prefix and/or a list of file extensions.

In [16]:
img_path = (path/'my_icons')
img_files=img_path.ls(file_type='image')
assert len(img_files) > 0 and img_files[0].suffix=='.gif'
ipy_files=path.ls(file_exts=['.ipynb'])
assert len(ipy_files) > 0 and ipy_files[0].suffix=='.ipynb'
img_files[0],ipy_files[0]

(Path('my_icons/atom.gif'), Path('02_utils.ipynb'))

In [17]:
pkl = pickle.dumps(path)
p2 =pickle.loads(pkl)
test_eq(path.ls()[0], p2.ls()[0])

In [18]:
@patch
def __repr__(self:Path):
    b = getattr(Path, 'BASE_PATH', None)
    if b:
        try: self = self.relative_to(b)
        except: pass
    return f"Path({self.as_posix()!r})"

fastai also updates the `repr` of `Path` such that, if `Path.BASE_PATH` is defined, all paths are printed relative to that path (as long as they are contained in `Path.BASE_PATH`:

In [19]:
t = ipy_files[0].absolute()
try:
    Path.BASE_PATH = t.parent.parent
    test_eq(repr(t), f"Path('_notebooks/{t.name}')")
finally: Path.BASE_PATH = None

In [22]:
def join_path_file(file, path, ext=''):
    "Return `path/file` if file is a string or a `Path`, file otherwise"
    if not isinstance(file, (str, Path)): return file
    path.mkdir(parents=True, exist_ok=True)
    return path/f'{file}{ext}'

In [24]:
import shutil

In [25]:
path = Path.cwd()/'_tmp'/'tst'
f = join_path_file('tst.txt', path)
assert path.exists()
test_eq(f, path/'tst.txt')
with open(f, 'w') as f_: assert join_path_file(f_, path) == f_
shutil.rmtree(Path.cwd()/'_tmp')